In [218]:
%matplotlib widget
# %pylab inline
from matplotlib.pylab import *
datafile = "data.npz"
import numpy as np

In [219]:
o = np.load(datafile)
im = o["image"]
upx, upy = o["upperlimit"]
lox, loy = o["lowerlimit"]

In [220]:
# figure(figsize=(10,10))
imshow(im)
plt.scatter(upy,upx, c='y', s=1)
plt.scatter(loy,lox, c='y', s=1)
# plt.scatter(loy[602],lox[602], c='r', s=50)
# plt.scatter(upy[602],upx[602], c='r', s=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [221]:
# we want to analyze a full segment of the layered body
start, end = 1400, 1640 # start and end column

In [222]:
figure(figsize=(10,10))
imshow(im)
plt.scatter(upy,upx, c='y', s=1)
plt.scatter(loy,lox, c='y', s=1)

vlines([start, end], 0, 1600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [225]:
from attr import ib, attrs
@attrs
class TimeSerie():
    signal = ib()
    start = ib()
    end = ib()
    column = ib()
    


In [226]:
# now we extract the timeseries, also a plot just to be sure of what we are doing

figure()
imshow(im)
gray()
series = []
noises = []

nlength = 150 # lenght of ts for noise
buf = 10

for i, colid in enumerate(np.arange(start, end)):
#     print(f"column {colid}")
    if colid not in upy:
#         print("not")
        continue
        
    col_pos_in_list = np.where(colid == upy)[0][0]
        
    astart = upx[col_pos_in_list]
    aend = lox[col_pos_in_list]
#     print(astart, aend)
    
    ts = im.T[colid][astart:aend]
    
    nstart = astart-nlength - buf
    nend = astart-buf
    
    no = im.T[colid][nstart:nend]
    plot(ts*0.1+colid, np.arange(len(ts))+astart+1, color="b", linewidth=0.5)
    
    series.append(TimeSerie(signal=ts, start=astart, end=aend, column=colid))
    noises.append(TimeSerie(signal=no, start=nstart, end=nend, column=colid))
    
# gca().set_ylim(gca().get_ylim()[::-1])

plt.scatter(upy,upx, c='y', s=1)
plt.scatter(loy,lox, c='y', s=1)

vlines([start, end], 0, 1600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [227]:
# test a wiggle plot, it highlights the horizons, often used in seismic data

figure()
for ts in series:
    
    s = ts.signal - np.mean(ts.signal)
    s /= np.std(s)
    
    plot(s*2+ts.column, np.arange(len(s))+ts.start, color="b", linewidth=0.3)
    
gca().set_ylim(gca().get_ylim()[::-1])


vlines([start, end], 0, 1600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [228]:
# test a wiggle plot of noise, it highlights the horizons, often used in seismic data, these are examples of noise

figure()
for ts in noises:
    
    s = ts.signal - np.mean(ts.signal)
    s /= np.std(s)
    
    plot(s*2+ts.column, np.arange(len(s))+ts.start, color="b", linewidth=0.3)
    
gca().set_ylim(gca().get_ylim()[::-1])


vlines([start, end], 0, 1600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [236]:
from scipy.signal import periodogram, get_window, detrend, find_peaks, find_peaks_cwt
padto =10000
powers = []

for n in noises:
    
    w = get_window("hamming", len(n.signal))
    
    signal = detrend(n.signal, type="linear")
    
    signal = signal*w
    
    padded = np.pad(signal, (0, padto-len(n.signal)))
    f, p = periodogram(padded, detrend="linear", fs =1) # fs should be set to a sampling frequency that makes sense e.g. time
    powers.append(p)
    
noise_f = f
avg_p_noise  = np.mean(np.row_stack(powers), axis=0) # the averagepower spectrum of the noise, to have a reference

In [237]:
ci95 = np.percentile(powers, 99.9, axis=0)

In [238]:
# now the same for the signals
padto =10000
powers = []
for n in series:
    w = get_window("hamming", len(n.signal))
    signal = detrend(n.signal, type="linear")
    
    signal = signal*w
    padded = np.pad(signal, (0, padto-len(n.signal)))
    
    f, p = periodogram(padded, detrend=False, fs =1) # fs should be set to a sampling frequency that makes sense e.g. time step of the radar signal
    powers.append(p)
    
signal_f = f
avg_p_signals  = np.mean(np.row_stack(powers), axis=0) # the average power spectrum of the noise, to have a reference

In [239]:
lengths = []
for n in series:
    lengths.append(len(n.signal))
    
import scipy
scipy.stats.describe(lengths) # we have a look at the lengths of the signals we are analyzing, theygo from 100 to 150, average is 132

DescribeResult(nobs=240, minmax=(112, 145), mean=132.09166666666667, variance=47.01248256624826, skewness=-0.7185013165205091, kurtosis=0.28940896942435357)

array([ 119,  435,  597,  723, 1209, 1583, 2028, 2173, 2633, 2845, 2962,
       3207, 3361, 3558, 3724, 3923, 4034, 4216, 4256, 4371, 4495, 4593,
       4926])

In [259]:
def plot_peaks(freqs, power):
    peaks = find_peaks(power)[0]
    tt = ['{:6.2f}'.format(1/val) for val in freqs[peaks]]

    for s, p, t in zip(freqs[peaks], power[peaks], tt):
        text(s,p,t)
        
    return peaks

In [260]:
figure()
plot(noise_f, avg_p_noise, label="noise")
plot(signal_f, avg_p_signals, label="signal")
plot(noise_f, ci95*20, label="C99.9")


scatter(signal_f[peaks], avg_p_signals[peaks])



plot_peaks(signal_f, avg_p_signals)
    
title("peaks in periods [samples]")
legend()
xlim(0, 0.20)

<ipython-input-260-afd0945db81b>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 0.2)

The peaks are at 84, 14, and 8, meaning good potion of the spectral energy is concentrated around these bands.
84 is not so far from the average lengths of the signals (134) making it impossible to establish if it is noise or not
but the 14/17 and 8 might be real. In particular around 8 might be the main layering, which would seem to be organized in boundles of largest length, around 15)
We now try to better attribute these features to the signal, as example we use the column 1530

In [261]:
for ts in series: # search the corresponding signal
    if ts.column == 1530:
        selected = ts
    

In [286]:
figure()
s = selected.signal
s = detrend(s, type="linear")
plot(s)

f, p = periodogram(s)
figure()
plot(f,p)

plot_peaks(f,p)

<ipython-input-286-21a9afadf06a>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-286-21a9afadf06a>:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([ 2,  4,  6,  8, 10, 13, 17, 20, 22, 26, 31, 33, 37, 42, 47, 51, 56,
       59, 63, 68])

We see that we have again 6, 8, 14 and 23 as main periods, might they be all harmonics? It would be interesting to create a fake signal to test what kind of harmonics one get out of them
we try to filter out these components to see what they represent

In [264]:
from scipy import fftpack

In [279]:
def fft_bandpass_filter(s, interval, time_step=1):
    signal_fft = fftpack.fft(s)

    # The corresponding frequencies
    freqs = fftpack.fftfreq(s.size, d=time_step)

    copy = np.zeros(freqs.size)

    low, high = interval
    slotfreqs = np.logical_and(np.abs(freqs) > low, np.abs(freqs) < high)
    copy[slotfreqs] = signal_fft[slotfreqs]

    filtered_signal = fftpack.ifft(copy)
    return filtered_signal

def fft_bandpass_filter_multibands(s, intervals, time_step = 1):
    signal_fft = fftpack.fft(s)

    # The corresponding frequencies
    freqs = fftpack.fftfreq(s.size, d=time_step)

    copy = np.zeros(freqs.size)

    for low, high in intervals:
        slotfreqs = np.logical_and(np.abs(freqs) > low, np.abs(freqs) < high)
        copy[slotfreqs] = signal_fft[slotfreqs]

    filtered_signal = fftpack.ifft(copy)
    return filtered_signal

In [287]:
ints = [(0.038, 0.048), (0.065, 0.085), (0.114, 0.133), (0.149, 0.175)]

f = fft_bandpass_filter_multibands(s, time_step=1, intervals=ints) #time step should be changed to something more meaningful, as earlier in this notebook

<ipython-input-279-77d9618a5c14>:26: ComplexWarning: Casting complex values to real discards the imaginary part
  copy[slotfreqs] = signal_fft[slotfreqs]


In [288]:
figure()

plot(s)

for i, inte in enumerate(ints):
    ss = fft_bandpass_filter(s, inte, 1 )
    plot(ss+i*20)

<ipython-input-288-9c0771ef3ea9>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-279-77d9618a5c14>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  copy[slotfreqs] = signal_fft[slotfreqs]
/usr/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
<ipython-input-279-77d9618a5c14>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  copy[slotfreqs] = signal_fft[slotfreqs]
/usr/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
<ipython-input-279-77d9618a5c14>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  copy[slotfreqs] = signal_fft[slotfreqs]
/usr/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=Fal

In [12]:
# we use an R package called astrochron for the speectral estimates, it might not be really straighforward
# install R, then open an R console by calling "R" in the command line
# now we install the librarby we will need "install.packages("astrochron")"
# might not work at all on windows


import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
numpy2ri.activate()


In [ ]:
# next to be finished/evaluated for the future

In [13]:
from rpy2.robjects.packages import importr
utils = importr('utils')
# utils.chooseCRANmirror(ind=1)
# utils.install_packages('astrochron')

In [14]:
from rpy2.robjects.packages import importr
ac = importr("astrochron")



In [37]:
s = series[10].signal

dat = np.column_stack([np.arange(len(s)), s])
power = ac.mtmML96(dat, linLog=2, output=1, genplot=False, detrend=True, demean=True, padfac=20)


----- PERFORMING Mann and Lees (1996) Robust Red Noise Analysis -----
 * Number of data points in stratigraphic series: 127 
 * Stratigraphic series length (space or time): 126 
 * Sampling interval (space or time): 1 
 * Will use default setting of 5 DPSS tapers
 * Mean value subtracted= 90.74803 
 * Linear trend subtracted. m= -0.1982272 b= 12.48831 
 * Estimated Conventional AR1 coefficient = 0.95527 
 * Nyquist frequency: 0.5 
 * Rayleigh frequency: 0.007874016 
 * MTM Power spectrum bandwidth resolution (halfwidth): 0.02362205 
 * Padded to 2540 points
 * Frequency grid spacing: 0.0003937008 
 * Calculating median smoothed spectrum using Tukey's robust end-point rule and symmetrical medians
 * Median smoothing window increased by 1 point
 * Number of smoothing points= 255 
 * Calculating analytic fit of AR1 to median smoothed spectrum using Brent's method
 * Estimated Robust AR1 coefficient = 0.9942724 

 * Searching for significant spectral peaks that satisfy 90 % CL
     requir

In [39]:
f = power["Frequency"]
p = power["Power"]
figure()
plot(f, p)
xlim(0, 0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 0.2)

In [36]:
ac.mtmML96?

Signature:       ac.mtmML96(*args, **kwargs)
Type:            DocumentedSTFunction
String form:    
function (dat, tbw = 3, ntap = NULL, padfac = 5, demean = T,
           detrend = F, medsmooth = 0.2, o <...> "
           return(sigfreq)
           }
           }
           <bytecode: 0x55bbce163f80>
           <environment: namespace:astrochron>
           
File:            /usr/lib/python3.8/site-packages/rpy2/robjects/functions.py
Docstring:      
Python representation of an R function.
description
-----------


 Mann and Lees (1996) robust red noise MTM analysis. This function implements several improvements to the algorithm used in SSA-MTM toolkit, including faster AR1 model optimization, and more appropriate 'edge-effect' treatment.
 


mtmML96(
    dat,
    tbw = 3.0,
    ntap = rinterface.NULL,
    padfac = 5.0,
    demean = <rpy2.rinterface.ListSexpVector object at 0x7f84063653c0> [RTYPES.VECSXP],
    detrend = <rpy2.rinterface.ListSexpVector object at 0x7f8406365dc0> [RTYPES